In [2]:
import cv2
import pandas as pd
import numpy as np
import joblib

In [3]:
# ── 1. Load model & encoder ──────────────────────────────────────────────────
best_model = joblib.load("best_gesture_model.pkl")
le = joblib.load("label_encoder.pkl")

In [ ]:
# ── 2. Load landmarks CSV ─────────────────────────────────────────────────────
df = pd.read_csv("/Users/ahmedtarek/Developer/Python/ML1 Project/video_landmarks.csv")

feature_cols = [c for c in df.columns if c != 'frame']   # drop frame column
X = df[feature_cols].copy()

x_cols = list(range(0, X.shape[1], 3))
y_cols = [c + 1 for c in x_cols]

# ── 3. Same normalization used during training ────────────────────────────────
X.iloc[:, x_cols] = X.iloc[:, x_cols].sub(X.iloc[:, 0], axis=0)
X.iloc[:, y_cols] = X.iloc[:, y_cols].sub(X.iloc[:, 1], axis=0)

# Scale using 4 fingertips (no thumb) — keypoints 8,12,16,20
fingertip_indices = [8, 12, 16, 20]

fingertip_distances = []
for tip in fingertip_indices:
    col_x = tip * 3        # ✅ 0-indexed: keypoint 8 → col 24
    col_y = tip * 3 + 1
    dist = np.sqrt(X.iloc[:, col_x]**2 + X.iloc[:, col_y]**2)
    fingertip_distances.append(dist)

div = np.mean(fingertip_distances, axis=0)
div = np.where(div < 1e-6, 1e-6, div)

X.iloc[:, x_cols] = X.iloc[:, x_cols].div(div, axis=0)
X.iloc[:, y_cols] = X.iloc[:, y_cols].div(div, axis=0)

new_cols = []
for i in range(21):
    new_cols += [f"x{i+1}", f"y{i+1}", f"z{i+1}"]
X.columns = new_cols

# ── 4. Predict labels ─────────────────────────────────────────────────────────
encoded_preds = best_model.predict(X)
labels        = le.inverse_transform(encoded_preds)

# Map frame_index → label  (one label per frame)
frame_labels = dict(zip(df['frame'], labels))

# ── 5. Write labelled video ───────────────────────────────────────────────────
cap = cv2.VideoCapture('/Users/ahmedtarek/Downloads/ITI/Machine Learning 1/Project/ml1.mov')

fps    = cap.get(cv2.CAP_PROP_FPS)
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

out = cv2.VideoWriter(
    "labelled_video.mp4",
    cv2.VideoWriter_fourcc(*"mp4v"),
    fps,
    (width, height)
)

frame_idx = 0
while True:
    success, frame = cap.read()
    if not success:
        break

    label = frame_labels.get(frame_idx, "no hand")

    cv2.putText(
        frame, label,
        (20, 50),
        cv2.FONT_HERSHEY_SIMPLEX,
        1.2, (0, 255, 0), 3
    )

    out.write(frame)
    frame_idx += 1

cap.release()
out.release()
print(f"Done — {frame_idx} frames written to labelled_video.mp4")

Done — 976 frames written to labelled_video.mp4
